In [1]:
import os
import random
import numpy as np
from tqdm import tqdm
import torch
from torch import nn, Tensor
import pdb
from typing import Optional, Tuple, Union
import logging
import math
from sklearn.metrics import accuracy_score

from transformers import TrainingArguments, Trainer, TrainerCallback, DefaultDataCollator
from transformers import AutoConfig, BigBirdTokenizer, BigBirdModel, BigBirdPreTrainedModel, BigBirdForMaskedLM, BigBirdForSequenceClassification
from transformers.modeling_outputs import SequenceClassifierOutput

# os.environ["TOKENIZERS_PARALLELISM"] = "true"


from accelerate import Accelerator, skip_first_batches
from accelerate import __version__ as accelerate_version
from accelerate.utils import DistributedDataParallelKwargs, GradientAccumulationPlugin

2023-07-25 17:15:23.535636: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 17:15:23.644997: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-25 17:15:24.089886: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.6/lib64:
2023-07-25 17:15:24.089955: W tensorflow/compiler/xla/strea

## Seed 고정

In [2]:
def seed_everything(seed:int = 1004):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # current gpu seed
    torch.cuda.manual_seed_all(seed) # All gpu seed
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = False  # True로 하면 gpu에 적합한 알고리즘을 선택함.

seed_everything(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Config

In [3]:
MODEL_NAME = 'google/bigbird-roberta-base'
MODEL_PATH = '/home/user/10TB/personalityAI/jonathn/BigBird_rand1'

In [4]:
config = AutoConfig.from_pretrained(MODEL_PATH)  

print(config)

BigBirdConfig {
  "_name_or_path": "/home/user/10TB/personalityAI/jonathn/BigBird_rand1",
  "architectures": [
    "BigBirdForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attention_type": "block_sparse",
  "block_size": 64,
  "bos_token_id": 1,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 4096,
  "model_type": "big_bird",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_random_blocks": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "rescale_embeddings": false,
  "sep_token_id": 66,
  "torch_dtype": "float32",
  "transformers_version": "4.29.2",
  "type_vocab_size": 2,
  "use_bias": true,
  "use_cache": true,
  "vocab_size": 50358
}



In [5]:
tokenizer = BigBirdTokenizer.from_pretrained(MODEL_NAME)  # 토크나이저는 학습되어 있는 것으로, BERTTOKENIZE

In [6]:
tokenizer

BigBirdTokenizer(name_or_path='google/bigbird-roberta-base', vocab_size=50358, model_max_length=4096, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

In [7]:
tokenizer("Hello I'm World!"), tokenizer.decode(tokenizer.encode("Hello I'm World!"))

({'input_ids': [65, 18536, 415, 1202, 2260, 101, 66], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]},
 "[CLS] Hello I'm World![SEP]")

In [8]:
tokenizer.model_max_length

4096

In [9]:
class MAEClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, num_labels):
        super().__init__()
        self.dense = nn.Linear(768, 768)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(768, num_labels)  # num_labels == 5
        self.gelu = NewGELUActivation()

    def forward(self, features, **kwargs):
        x = features[:, 0, :]  # take <s> token (equiv. to [CLS])
        x = self.dropout(x)
        x = self.dense(x)
        x = self.gelu(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        
        return x

In [10]:
class NewGELUActivation(nn.Module):
    """
    Implementation of the GELU activation function currently in Google BERT repo (identical to OpenAI GPT). Also see
    the Gaussian Error Linear Units paper: https://arxiv.org/abs/1606.08415
    """

    def forward(self, input: Tensor) -> Tensor:
        return 0.5 * input * (1.0 + torch.tanh(math.sqrt(2.0 / math.pi) * (input + 0.044715 * torch.pow(input, 3.0))))

In [11]:
class BigBirdForCustomClassification(BigBirdPreTrainedModel):
    def __init__(self, config, num_labels):
        super().__init__(config)
        self.num_labels = num_labels
        self.config = config
        self.bert = BigBirdModel(config)
        self.classifier = MAEClassificationHead(self.num_labels)

        # Initialize weights and apply final processing
        self.post_init()


    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):

        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]
        logits = self.classifier(sequence_output)
        
        loss = None
        loss_fct = nn.L1Loss()
        
       # loss_fct = loss_fct(logits_O.view(-1, self.num_labels), labels[0])
        loss = loss_fct(logits.squeeze(), labels.squeeze())

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states,attentions=outputs.attentions)

In [12]:
model = BigBirdForCustomClassification.from_pretrained(MODEL_PATH, 5)

print(model)

Some weights of the model checkpoint at /home/user/10TB/personalityAI/jonathn/BigBird_rand1 were not used when initializing BigBirdForCustomClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BigBirdForCustomClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForCustomClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForCustomClassification were not initialized from the model che

BigBirdForCustomClassification(
  (bert): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(50358, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0): BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdBlockSparseAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       

In [13]:
print(model.num_parameters())

128063237


In [14]:
from torchinfo import summary

summary(model)

Layer (type:depth-idx)                                            Param #
BigBirdForCustomClassification                                    --
├─BigBirdModel: 1-1                                               --
│    └─BigBirdEmbeddings: 2-1                                     --
│    │    └─Embedding: 3-1                                        38,674,944
│    │    └─Embedding: 3-2                                        3,145,728
│    │    └─Embedding: 3-3                                        1,536
│    │    └─LayerNorm: 3-4                                        1,536
│    │    └─Dropout: 3-5                                          --
│    └─BigBirdEncoder: 2-2                                        --
│    │    └─ModuleList: 3-6                                       85,054,464
│    └─Linear: 2-3                                                590,592
│    └─Tanh: 2-4                                                  --
├─MAEClassificationHead: 1-2                                    

## 데이터 준비

In [15]:
import pandas as pd

FI_train = pd.read_csv("/home/user/10TB/Data/FI/FI_train_pp.csv", index_col = 0)
FI_val = pd.read_csv("/home/user/10TB/Data/FI/FI_val_pp.csv", index_col = 0)
FI_test = pd.read_csv("/home/user/10TB/Data/FI/FI_test_pp.csv", index_col = 0)

In [ ]:
FI_train['label'] = FI_train[['openness', 'conscientiousness', 
                              'extraversion', 'agreeableness', 
                              'neuroticism']].apply(
    lambda row: ' '.join(row.values.astype('str')), axis=1)

FI_val['label'] = FI_val[['openness', 'conscientiousness', 
                              'extraversion', 'agreeableness', 
                              'neuroticism']].apply(
    lambda row: ' '.join(row.values.astype('str')), axis=1)

FI_test['label'] = FI_test[['openness', 'conscientiousness', 
                              'extraversion', 'agreeableness', 
                              'neuroticism']].apply(
    lambda row: ' '.join(row.values.astype('str')), axis=1)

FI_train

In [17]:
class bigbird_Dataset(torch.utils.data.Dataset):
    def __init__(self, data, label, tokenizer):  # 전처리된 데이터 셋이 들어옴
        self.data = data
        self.tokenizer = tokenizer
        self.label = label

    def __getitem__(self, idx):
        # gradient 계산에 영향을 주지 않게 clone().detach() 실행
        
        text = self.data[idx]
        tokens = self.tokenizer(text, 
                                #return_tensors="pt",  # pytorch.Tensor로 리턴
                                max_length=1024, 
                                padding="max_length",  
                                truncation=True,  # max_length 넘어가면 버림)
                               )
        # tokens['label'] = torch.FloatTensor([self.label[idx]])
        tokens['label'] = [float(i) for i in self.label[idx].split()]
        
        return tokens

    def __len__(self):  # 샘플 수
        return len(self.data)
    
    """
    class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, label):  # 전처리된 데이터 셋이 들어옴
        self.dataset = dataset
        self.label = label

    def __getitem__(self, idx):
        # gradient 계산에 영향을 주지 않게 clone().detach() 실행
        
        item = {key: val[idx].clone().detach() for key, val in self.dataset.items()}
        item['label'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):  # 샘플 수
        return len(self.label)
    """

In [18]:
train_sen = FI_train['Script'].tolist()
train_label = FI_train['label'].tolist()

val_sen = FI_val['Script'].tolist()
val_label = FI_val['label'].tolist()

test_sen = FI_test['Script'].tolist()
test_label = FI_test['label'].tolist()

In [19]:
train_dataset = bigbird_Dataset(train_sen, train_label, tokenizer)
val_dataset = bigbird_Dataset(val_sen, val_label, tokenizer)
test_dataset = bigbird_Dataset(test_sen, test_label, tokenizer)

In [20]:
print(train_dataset.__len__())
print(train_dataset.__getitem__(970))
#print(train_dataset.__getitem__(970)['input_ids'].numpy())
# print(tokenizer.decode(train_dataset.__getitem__(970)['input_ids'].numpy()[0]))

5902
{'input_ids': [65, 2242, 415, 3353, 430, 717, 1305, 131, 18679, 508, 1968, 439, 635, 4105, 1742, 10013, 6862, 131, 415, 993, 363, 936, 457, 1381, 775, 11070, 34274, 1784, 1302, 415, 474, 358, 5242, 745, 4426, 419, 415, 3606, 717, 3498, 2383, 467, 179, 5669, 446, 561, 1381, 631, 102, 66, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [21]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import multilabel_confusion_matrix
from transformers import EvalPrediction
from sklearn.metrics import mean_absolute_error
import torch

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions
    
    MAE = 1 - mean_absolute_error(labels, preds)
    
    
    return {
        '1 - MAE' : MAE
    }

In [22]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        
        outputs = model(**inputs)
        
        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            if unwrap_model(model)._get_name() in MODEL_FOR_CAUSAL_LM_MAPPING_NAMES.values():
                loss = self.label_smoother(outputs, labels, shift_labels=True)
            else:
                loss = self.label_smoother(outputs, labels)
        else:
            if isinstance(outputs, dict) and "loss" not in outputs:
                raise ValueError(
                    "The model did not return a loss from the inputs, only the following keys: "
                    f"{','.join(outputs.keys())}. For reference, the inputs it received are {','.join(inputs.keys())}."
                )
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

        return (loss, outputs) if return_outputs else loss

In [23]:
training_args = TrainingArguments(
    output_dir="/home/user/10TB/personalityAI/FI_1head/RS1-5_FreezeBigBird_FT_1-head_mae",
    logging_dir= "/home/user/10TB/personalityAI/FI_1head/RS1-5_FreezeBigBird_FT_1-head_mae_log",
    num_train_epochs=10,
    learning_rate = 1e-5,
   # max_steps=1000,
    per_device_train_batch_size=8,
#    gradient_accumulation_steps = 16,
    per_device_eval_batch_size = 8,
#    eval_accumulation_steps = 32,
    logging_strategy = "epoch",
    save_strategy = "epoch",
    lr_scheduler_type = "linear",
    dataloader_num_workers = 12,
    warmup_ratio = 0.1,
    weight_decay=0.01,
    evaluation_strategy='epoch',
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=DefaultDataCollator(return_tensors = "pt"),
    compute_metrics=compute_metrics,
)

In [25]:
#trainer.add_callback(CustomCallback(trainer))
trainer.train()

/home/egg2018037024/venv/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/egg2018037024/venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,1 - mae
1,0.356500,0.128738,0.871262
2,0.135700,0.119913,0.880087
3,0.129400,0.117508,0.882492
4,0.127000,0.116428,0.883572
5,0.126200,0.115377,0.884623
6,0.124900,0.114759,0.885241
7,0.123700,0.114755,0.885245
8,0.123500,0.114316,0.885684
9,0.123400,0.114190,0.885810
10,0.123000,0.114218,0.885782


/home/egg2018037024/venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/egg2018037024/venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/egg2018037024/venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/egg2018037024/venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=1850, training_loss=0.14934009242702176, metrics={'train_runtime': 2408.6136, 'train_samples_per_second': 24.504, 'train_steps_per_second': 0.768, 'total_flos': 3.127262534602752e+16, 'train_loss': 0.14934009242702176, 'epoch': 10.0})

## Test

In [26]:
model = BigBirdForCustomClassification.from_pretrained("/home/user/10TB/personalityAI/FI_1head/RS1-5_FreezeBigBird_FT_1-head_mae/checkpoint-1665", 5)

In [27]:
def test_compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions
    
    labels = labels.T
    preds = preds.T
    
    MAE = []
    
    for i in range(5):
        MAE.append(1 - mean_absolute_error(labels[i], preds[i]))
    
    
    return {
        '1 - MAE' : MAE
    }

In [28]:
eval_trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DefaultDataCollator(return_tensors = "pt"),
    compute_metrics=test_compute_metrics,
)

In [29]:
eval_trainer.evaluate(test_dataset)

/home/egg2018037024/venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Trainer is attempting to log a value of "[0.885399580001831, 0.8818990141153336, 0.8807056471705437, 0.8948393911123276, 0.8803003504872322]" of type <class 'list'> for key "eval/1 - MAE" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.11535132676362991,
 'eval_1 - MAE': [0.885399580001831,
  0.8818990141153336,
  0.8807056471705437,
  0.8948393911123276,
  0.8803003504872322],
 'eval_runtime': 29.2131,
 'eval_samples_per_second': 67.435,
 'eval_steps_per_second': 2.122}